<a href="https://colab.research.google.com/github/Elwing-Chou/tibaml1027/blob/main/imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84140032/84125825 [==============================] - 2s 0us/step


In [4]:
dataset

'/root/.keras/datasets/aclImdb.tar.gz'

In [2]:
import os
import glob
import pandas as pd
def getdata(mid):
    dn = os.path.dirname(dataset)
    posfn = glob.glob(os.path.join(dn, "aclImdb", mid, "pos", "*"))
    negfn = glob.glob(os.path.join(dn, "aclImdb", mid, "neg", "*"))
    contents = []
    for fn in posfn + negfn:
        with open(fn, encoding="utf-8") as f:
            contents.append(f.read())
    df = pd.DataFrame({
        "content":contents,
        "sentiment":[1] * len(posfn) + [0] * len(negfn)
    })
    return df

In [5]:
train_df = getdata("train")
test_df = getdata("test")

In [7]:
test_df

,content,sentiment
0,"Valentine ""Dogkiller"" Dussaut and Joe ""The Jud...",1
1,Ruthless mercenary Bruno Rivera (Paul Naschy i...,1
2,Should we take the opening shot as a strange f...,1
3,"Being a retired medical/health field ""toiler i...",1
4,I wasn't expecting a great deal from this film...,1
...,...,...
24995,"I give it a 2, because of the beautiful Medite...",0
24996,How bad can you make a film. A good question w...,0
24997,"Oh, my. Oh, this is a *really* bad movie. The ...",0
24998,Apparently re-cut episodes from the Gangbuster...,0


In [13]:
# Tokenize: 把你的詞變成數字
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
tok.fit_on_texts(train_df["content"])

In [ ]:
# tok.word_index
# tok.index_word

In [14]:
# Sequence: 化成數字的序列
x_train_seq = tok.texts_to_sequences(train_df["content"])
x_test_seq = tok.texts_to_sequences(test_df["content"])

In [20]:
INPUT_LENGTH = 512
INPUT_DIM = 3000
OUTPUT_DIM = 128

In [ ]:
# pd.DataFrame(x_train_seq)
# Padding: 截長補短變成一樣長
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, maxlen=INPUT_LENGTH)
x_test_pad = pad_sequences(x_test_seq, maxlen=INPUT_LENGTH)
pd.DataFrame(x_train_pad)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
layers = [
    # 3001(種詞) * 128(個情緒)
    Embedding(INPUT_DIM+1, OUTPUT_DIM, mask_zero=True, input_length=INPUT_LENGTH),
    Flatten(),
    Dense(256, activation="relu"),
    Dropout(0.25),
    Dense(2, activation="softmax")
]
model = Sequential(layers)
model.summary()

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
layers = [
    # 3001(種詞) * 128(個情緒)
    Embedding(INPUT_DIM+1, OUTPUT_DIM, mask_zero=True, input_length=INPUT_LENGTH),
    GlobalAveragePooling1D(),
    Dense(2, activation="softmax")
]
model = Sequential(layers)
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 512, 128)          384128    
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_4 (Dense)             (None, 2)                 258       
                                                                 
Total params: 384,386
Trainable params: 384,386
Non-trainable params: 0
_________________________________________________________________
